h(x,y) = (x^2 * sin y^2) + (x + y)

-1 ≤ x ≤ 2 dan -1 ≤ y ≤ 1

In [2]:
import random
import math
import pandas as pd

# Fungsi Bantuan

In [3]:
# Membuat Kromosom Populasi

def generatePopulation(sum_pop, bitnum) :
    pops = []
    for i in range (sum_pop) :
        chromosomes = []
        for i in range (bitnum) :
            chromosomes.append(random.choice([0,1]))
        pops.append(chromosomes)
    return pops

In [4]:
# Mengubah X dan Y dari Biner menjadi Float

def convert(bin_x, bitnum, ra, rb) :
    ret = []
    a = ra - rb
    b = 0
    for i in range (1, bitnum+1) :
        b += 2 ** (i*-1)
    c = 0
    for i in range (len(bin_x)) :
        temp = 0
        c = 0
        for j in range (bitnum) :
            temp = (j+1)*-1

            c += bin_x[i][j] * (2**temp)
        sum = rb + ((a / b) * c)
        ret.append(sum)
    return ret


In [5]:
# Menggabungkan kromosom X dan Y

def combineChrom(bin_x,bin_y) :
  new_population = []
  for i in range(len(bin_x)):
    new_population.append(bin_x[i] + bin_y[i])

  return new_population

In [6]:
# Menghitung nilai fitness dari X dan Y

def computeFitness(dec_x, dec_y) :
  ret = []
  for i in range(len(dec_x)) :
      x = dec_x[i]
      y = dec_y[i]
      sum = (x**2 * math.sin(y**2)) + (x+y)
      ret.append(sum)
  
  return ret

# Seleksi Orang tua dan Crossover

In [7]:
# seleksi orangtua

def ParentSelRoulette(new_population,arr_fit,sum_pop):
  sumFit = 0
  getChance = []
  sumChance = []
  #Total Fitness
  
  for i in range(sum_pop):
    sumFit += arr_fit[i]
  #Normalisasi Fitness / Persentase 
  for i in range(sum_pop):
    getChance.append(arr_fit[i] / sumFit)
  #Total Normalisasi Fitness
  sumFit = 0
  for i in range(sum_pop):
    sumFit += getChance[i]
    sumChance.append(sumFit)

  Parent = []
  for i in range(2):
    rand = random.uniform(0,sumFit)
    for j in range(sum_pop):
      if sumChance[j] > rand :
        Parent.append(new_population[j])
      if len(Parent) == 2:
        return Parent[i-1],Parent[i] 

  return Parent[0],Parent[1]

In [8]:
#crossover
def crossOver(Parent1,Parent2):
  child1,child2 = [],[]
  prob = random.random()
  if prob < 0.9 :
    Halfing = random.randint(0,len(Parent1)-1)
    child1[:Halfing],child1[Halfing:] = Parent1[:Halfing],Parent2[Halfing:]
    child2[:Halfing],child2[Halfing:] = Parent2[:Halfing],Parent1[Halfing:]
    
    return child1,child2 
  
  else :
    return Parent1,Parent2

In [9]:
# Mencari nilai fitness maksimum
def findMaxFitness(fit_xy) :
    max = fit_xy[0]
    for i in range(1, len(fit_xy)-1) :
        if fit_xy[i] > max :
            max = fit_xy[i]
    return max

In [10]:
# Mencari 2 index fitness tertinggi
def find2IdxMaxFitness(fit_xy) :
    max1 = 0
    max2 = -1
    max = fit_xy[0]
    for i in range(1, len(fit_xy)-1) :
        if fit_xy[i] > fit_xy[max1] :
            max1 = i

    if max1 == 0 :
        max2 = 1
    else :
        max2 = 0
    for i in range(1, len(fit_xy)-1) :
        if fit_xy[i] > fit_xy[max2] and i != max1 :
            max2 = i

    return max1,max2

In [11]:
# Perhitungan untuk mutasi

def mutation (chromosome) :
    newChromosome = chromosome
    rand = random.randint(1,10)
    if rand == 1 :
        rand2 = random.randint(0, len(chromosome) - 1)
        if newChromosome[rand2] == 0 :
            newChromosome[rand2] = 1
        else :
            newChromosome[rand2] = 0
    return newChromosome

In [12]:
# Membuat generasi baru

def regeneration(xy, fit_xy, sum_pop) :
    newGen = []
    max1, max2 = find2IdxMaxFitness(fit_xy)
    newGen.append(xy[max1])
    newGen.append(xy[max2])
    for i in range ((sum_pop//2)-1) :
        parents1,parents2 = ParentSelRoulette(xy,fit_xy,sum_pop)
        childs1,childs2 = crossOver(parents1,parents2)
        childs1 = mutation(childs1)
        childs2 = mutation(childs2)
        newGen.append(childs1)
        newGen.append(childs2)
    return newGen

In [13]:
def split(chrom,check=False):
  
  split = len(chrom) // 2

  if check == True :
    return chrom[:split]
  return  chrom[split:]

# Fungsi Main

In [15]:
# Parameter umum
sum_pop = 20
bitnum = 3
raX = 2
raY = 1
rbXY = -1

# Variabel variabel penting

bin_x = generatePopulation(sum_pop, bitnum)
bin_y = generatePopulation(sum_pop, bitnum)
dec_x = convert(bin_x, bitnum, raX, rbXY)
dec_y = convert(bin_y, bitnum, raY, rbXY)
fit_xy = computeFitness(dec_x, dec_y)


# Proses
xy = combineChrom(bin_x, bin_y)
best_fit = []
best_chrome = []
best_fit.append(findMaxFitness(fit_xy))
for i in range (50) :
    next_xy = regeneration(xy,fit_xy, sum_pop)
    fit_xy = []
    next_x = []
    next_y = []
    for j in range (sum_pop) :
        next_x.append(split((next_xy[j]),check=True))
        next_y.append(split((next_xy[j])))
    next_dec_x = convert(next_x, bitnum, raX, rbXY)
    next_dec_y = convert(next_y, bitnum, raY, rbXY)
    fit_xy = computeFitness(next_dec_x, next_dec_y)
    xy = next_xy
    a, b = find2IdxMaxFitness(fit_xy)
    best_chrome.append(a)
    best_fit.append(findMaxFitness(fit_xy))

best_xy, temp = find2IdxMaxFitness(fit_xy)

# Output solusi terbaik
print("best solution : ", findMaxFitness(fit_xy))
print("chromosome : ", xy[best_xy])
print("x = ", next_dec_x[best_xy])
print("y = ", next_dec_y[best_xy])

best solution :  6.365883939231586
chromosome :  [1, 1, 1, 1, 1, 1]
x =  2.0
y =  1.0
